<a href="https://colab.research.google.com/github/shailesh008/FacialKeypointRecognition/blob/master/FacialKeyPoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import math
import cv2                     # OpenCV library for computer vision
from PIL import Image
import time

In [0]:
image = cv2.imread('images/test_image_1.jpg')

# Convert the image to RGB colorspace
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Plot our image using subplots to specify a size and title
fig = plt.figure(figsize = (8,8))
ax1 = fig.add_subplot(111)
ax1.set_xticks([])
ax1.set_yticks([])

ax1.set_title('Original Image')
ax1.imshow(image)

In [3]:
from utils import *

# Load training set
X_train, y_train = load_data()
print("X_train.shape == {}".format(X_train.shape))
print("y_train.shape == {}; y_train.min == {:.3f}; y_train.max == {:.3f}".format(
    y_train.shape, y_train.min(), y_train.max()))

# Load testing set
X_test, _ = load_data(test=True)
print("X_test.shape == {}".format(X_test.shape))

NameError: ignored

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure(figsize=(20,20))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)
for i in range(9):
    ax = fig.add_subplot(3, 3, i + 1, xticks=[], yticks=[])
    plot_data(X_train[i], y_train[i], ax)

In [0]:
# Import deep learning resources from Keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout
from keras.layers import Flatten, Dense


## TODO: Specify a CNN architecture
# Your model should accept 96x96 pixel graysale images in
# It should have a fully-connected output layer with 30 values (2 for each facial keypoint)

model = Sequential()

# Define your architecture.
model.add(Conv2D(filters=32, kernel_size=3, activation='relu', 
                        input_shape=(96, 96, 1)))

model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(212, activation='relu'))
model.add(Dense(30))

# Summarize the model
model.summary()

In [0]:
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam
from keras.callbacks import ModelCheckpoint 

## TODO: Compile the model
model.compile(Adamax(), loss='mean_squared_error', metrics=['accuracy'])

# Save the model weights
checkpointer = ModelCheckpoint(filepath='saved_models/weights_facial_keypoints.h5', 
                               verbose=1, save_best_only=True)

## TODO: Train the model
history = model.fit(X_train, y_train, 
          validation_split=0.2,
          epochs=50, batch_size=20, callbacks=[checkpointer], verbose=1)

## TODO: Save the model as model.h5
model.save('my_model.h5')

In [0]:
## TODO: Visualize the training and validation loss of your neural network
model.load_weights('./my_model.h5')

# Plotting
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [0]:
y_test = model.predict(X_test)
fig = plt.figure(figsize=(20,20))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)
for i in range(9):
    ax = fig.add_subplot(3, 3, i + 1, xticks=[], yticks=[])
    plot_data(X_test[i], y_test[i], ax)

In [0]:
# Load in color image for face detection
image = cv2.imread('images/obamas4.jpg')

# Convert the image to RGB colorspace
image_copy = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# plot the image
fig = plt.figure(figsize = (9,9))
ax1 = fig.add_subplot(111)
ax1.set_xticks([])
ax1.set_yticks([])
ax1.set_title('image copy')
ax1.imshow(image_copy)

In [0]:
### TODO: Use the face detection code we saw in Section 1 with your trained conv-net 
## TODO : Paint the predicted keypoints on the test image
# Extract the pre-trained face detector from an xml file
face_cascade = cv2.CascadeClassifier('detector_architectures/haarcascade_frontalface_default.xml')

# Convert the RGB  image to grayscale
gray = cv2.cvtColor(image_copy, cv2.COLOR_RGB2GRAY)

# Detect the faces in image
faces = face_cascade.detectMultiScale(gray, 1.25, 6)

# Make a copy of the orginal image to draw face detections on
image_with_detections = np.copy(image_copy)

for(x,y,w,h) in faces:
     # Add a red bounding box to the detections image
    cv2.rectangle(image_with_detections, (x,y), (x+w,y+h), (255,0,0), 3)

# plot our image
fig = plt.figure(figsize = (9,9))
ax1 = fig.add_subplot(111)
ax1.set_xticks([])
ax1.set_yticks([])
ax1.set_title('image_with_detections')
ax1.imshow(image_with_detections)

In [0]:
cropped_faces = []
for(x,y,w,h) in faces:
    # crop the section where the face is located
    face = gray[y:y+h,x:x+w]
    # resize and normalize
    face = cv2.resize(face, (96, 96)) / 255
    # Need to expand because CNN expects conv2d_1_input to have 4 dimensions
    face = np.expand_dims(face, axis=2)
    cropped_faces.append(face)

# load model
model = load_model('./my_model.h5')
X_faces = np.asarray(cropped_faces)
y_faces = model.predict(X_faces)

fig = plt.figure(figsize=(20,20))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)
for i in range(len(faces)):
    ax = fig.add_subplot(3, 3, i + 1, xticks=[], yticks=[])
    plot_data(X_faces[i], y_faces[i], ax)

In [0]:
# Plot the Facial Keypoints on the Original image

# plot the image
fig = plt.figure(figsize = (9,9))
ax1 = fig.add_subplot(111)
ax1.set_xticks([])
ax1.set_yticks([])
ax1.set_title('Original Image')

for i in range(len(faces)):
    x, y, w, h = faces[i]
    # rescaling back to original color image
    x_keypoints = y_faces[i][0::2] * 48 + 48
    x_scaled = (x_keypoints * w / 96) + x
    y_keypoints = y_faces[i][1::2] * 48 + 48
    y_scaled = (y_keypoints * h / 96) + y

    ax1.scatter(x_scaled, y_scaled, marker='.', c='b', s=20)

ax1.imshow(image_copy)